In [21]:
import pandas as pd;
import numpy as np;
import scipy as sp;
import sklearn;
import sys;
from nltk.corpus import stopwords;
import nltk;
from gensim.models import ldamodel
import gensim.corpora;
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer;
from sklearn.decomposition import NMF;
from sklearn.preprocessing import normalize;
from gensim import corpora, models
import pickle;

In [22]:
import csv
import re
#import spacy
import sys
import requests
import pandas as pd
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
import os
import sys, getopt
import numpy as np
from bs4 import BeautifulSoup
import urllib3
from glob import glob
from string import punctuation

from nltk.corpus import wordnet

def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)

def find_pdfs():
    return glob(os.path.join('./AllSyllabiParser',"*.{}".format('pdf')))

def convert(fname, pages=None):
    """ Function converting pdf to string """
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb')
    
    try:
        for page in PDFPage.get_pages(infile, pagenums):
            interpreter.process_page(page)
    except PDFTextExtractionNotAllowed:
        print('This pdf won\'t allow text extraction!')
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return strip_punctuation(text)

def word_count(string):
    my_string = string.lower().split()
    my_dict = {}
    for item in my_string:
        if item in my_dict:
            my_dict[item] += 1
        else:
            my_dict[item] = 1
    


In [23]:
list_of_pdfs = find_pdfs()
syllabus_string = ''
bag_of_words_all_syllabi = []
corpus = []
#Converting pdf to string
for pdf in list_of_pdfs:
    syllabus_string = convert(pdf)
    corpus.append(syllabus_string)
#     bag_of_words_one_syllabus = word_count(syllabus_string)
#     bag_of_words_all_syllabi.append(bag_of_words_one_syllabus)

This pdf won't allow text extraction!
This pdf won't allow text extraction!


In [24]:
# clean text

from nltk.corpus import words
from nltk.corpus import stopwords
 

allEnglishWords = set(words.words())
stopwordsList = set(stopwords.words('english'))

def clean_doc(doc, english = False, return_list = False, stopwords = False):
    listOfWords = re.findall(r'[^\d\W]{2,}', doc)
    listOfWords = [word.lower() for word in listOfWords]
    if english:
        listOfWords = [word for word in listOfWords if word in allEnglishWords]
    if stopwords:
        listOfWords = [word for word in listOfWords if word not in stopwordsList]
    if return_list:
        return listOfWords
    else:
        return " ".join(listOfWords)


In [25]:
cleaned_corpus = [clean_doc(doc, return_list = True, stopwords = True) for doc in corpus]

In [26]:
cleaned_corpus_1 = [clean_doc(doc, english = True, return_list = True, stopwords = True) for doc in corpus]

In [27]:
cleaned_corpus_2 = [clean_doc(doc, english = True, return_list = False, stopwords = True) for doc in corpus]

In [28]:
cleaned_corpus_3 = [clean_doc(doc, english = False, return_list = False, stopwords = True) for doc in corpus]

In [29]:
cleaned_corpus_4 = [clean_doc(doc, english = False, return_list = True, stopwords = True) for doc in corpus]

In [30]:
vectorizer = CountVectorizer(analyzer='word', stop_words='english', lowercase=True, token_pattern=r'[^\d\W]{2,}');
x_counts = vectorizer.fit_transform(cleaned_corpus_2);

In [31]:
words = vectorizer.get_feature_names()
words

['aa',
 'abdomen',
 'abduction',
 'aberration',
 'abide',
 'ability',
 'able',
 'abnormal',
 'abreast',
 'abroad',
 'absence',
 'absent',
 'absenteeism',
 'absolute',
 'absolutely',
 'absorb',
 'absorption',
 'abstract',
 'abstracted',
 'abstraction',
 'abstractly',
 'abundance',
 'abundant',
 'abuse',
 'academic',
 'academically',
 'academy',
 'accelerate',
 'accelerated',
 'acceleration',
 'accelerometer',
 'accept',
 'acceptable',
 'acceptance',
 'accepted',
 'acceptor',
 'access',
 'accessibility',
 'accessible',
 'accessory',
 'accident',
 'accidental',
 'accommodate',
 'accommodation',
 'accompany',
 'accomplish',
 'accomplished',
 'accordance',
 'according',
 'accordingly',
 'account',
 'accountability',
 'accountable',
 'accounting',
 'accreditation',
 'accrual',
 'accrue',
 'accumulate',
 'accumulation',
 'accuracy',
 'accurate',
 'accurately',
 'accustomed',
 'acetylation',
 'achieve',
 'achievement',
 'acid',
 'acidification',
 'acoustical',
 'acquaint',
 'acquainted',
 'acq

In [32]:
id2word = {}
for i, word in enumerate(words):
    id2word[i] = word

In [33]:
transformer = TfidfTransformer(smooth_idf=False);
x_tfidf = transformer.fit_transform(x_counts);

In [34]:
xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)

In [35]:
def get_nmf_topics(model, n_top_words):
    
    #the word ids obtained need to be reverse-mapped to the words so we can print the topic names.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(num_topics):
        
        #for each topic, obtain the largest values, and add the words they map to into the dictionary.
        words_ids = model.components_[i].argsort()[:-20 - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [36]:
num_topics = 25
model = NMF(n_components=num_topics, init='nndsvd')
model.fit(xtfidf_norm)

NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0,
  max_iter=200, n_components=25, random_state=None, shuffle=False,
  solver='cd', tol=0.0001, verbose=0)

In [37]:
get_nmf_topics(model, 20)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 16,Topic # 17,Topic # 18,Topic # 19,Topic # 20,Topic # 21,Topic # 22,Topic # 23,Topic # 24,Topic # 25
0,class,electricity,language,lab,art,history,physics,trigonometric,lesson,design,...,energy,space,inflation,architectural,statistics,media,health,vex,calculus,geography
1,student,lesson,culture,lecture,works,historical,magnetism,ti,medical,engineering,...,lesson,flight,economic,architecture,probability,video,justice,python,derivative,human
2,chapter,electronics,presentational,biology,studio,question,electric,trigonometry,body,project,...,sustainable,lesson,aggregate,lesson,sampling,audio,public,registered,differential,psychology
3,course,digital,interpersonal,laboratory,artistic,argument,electricity,polar,disease,element,...,water,aeronautics,supply,design,dogs,web,social,resume,graph,exam
4,assignment,circuit,interpretive,exam,portfolio,explain,course,angle,patient,process,...,alternative,aircraft,demand,green,joint,converting,course,studio,calculator,population
5,instructor,magic,communication,genetics,dimensional,evidence,motion,exponential,human,solution,...,impact,moon,economics,sustainable,data,element,oer,inventor,algebraic,environmental
6,grade,design,audio,organic,drawing,relative,laboratory,triangle,investigate,modeling,...,lead,design,unemployment,commercial,statistical,course,socioeconomic,design,function,placement
7,canvas,trip,cultural,ecology,ce,significance,scientific,algebra,murder,lesson,...,renewable,travel,money,ga,hip,use,unequal,project,logarithmic,course
8,discussion,basic,print,course,work,course,mechanics,logarithmic,illness,unit,...,way,propulsion,market,shipping,blocking,delivery,psychology,lead,exponential,score
9,week,electronic,respond,anatomy,content,world,introductory,inverse,crime,lead,...,thermostat,traveling,fiscal,construction,random,linking,status,based,theorem,advanced


**LDA**

In [38]:
id2word = gensim.corpora.Dictionary(cleaned_corpus_1);
id2word

In [39]:
bow_corpus = [id2word.doc2bow(doc) for doc in cleaned_corpus_1]

In [40]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [41]:
corpus_tfidf

In [42]:
lda = gensim.models.ldamodel.LdaModel(bow_corpus, num_topics=20, id2word=id2word, passes=50)

In [43]:
lda_tfidf = gensim.models.ldamodel.LdaModel(corpus_tfidf, num_topics=10, id2word=id2word, passes=50)

In [44]:
def get_lda_topics(model, num_topics):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = 20);
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict);

In [45]:
get_lda_topics(lda, 20)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,Topic # 11,Topic # 12,Topic # 13,Topic # 14,Topic # 15,Topic # 16,Topic # 17,Topic # 18,Topic # 19,Topic # 20
0,lab,class,language,calculus,anthropology,art,course,course,government,student,course,due,lab,design,class,course,space,lesson,lesson,analysis
1,course,course,course,cancer,class,works,class,exam,political,class,student,quiz,lecture,lesson,course,academic,aeronautics,body,days,course
2,laboratory,student,culture,smith,human,work,student,college,politics,course,class,complete,course,engineering,economic,student,living,human,activity,exam
3,may,work,communication,use,evolution,content,exam,history,us,instructor,college,course,exam,project,exam,learning,flight,medical,project,composition
4,must,week,exam,treatment,article,ce,may,historical,chapter,policy,instructor,chapter,class,unit,textbook,disability,moon,investigate,data,environmental
5,lecture,please,audio,mike,primate,making,work,score,system,attendance,learning,student,student,course,student,instructor,travel,explore,create,circuit
6,instructor,assignment,interpersonal,physical,physical,artistic,instructor,question,one,may,exam,psychology,may,process,policy,math,history,disease,computer,music
7,chapter,must,presentational,part,course,chemical,use,physics,read,participation,may,discussion,cell,way,history,test,science,health,problem,college
8,exam,grade,college,genetic,skeleton,global,learning,advanced,comparative,academic,use,psych,biology,page,supply,work,traveling,heart,way,science
9,academic,day,study,used,biological,architecture,days,placement,exam,office,information,week,grade,lead,read,policy,test,way,course,writing


In [46]:
get_lda_topics(lda_tfidf, 10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,lesson,constitution,presentational,hybrid,asp,duiker,electric,python,class,terminology
1,architecture,politics,interpretive,mail,marketing,nuclear,magnetism,game,lab,male
2,circuit,cancer,german,painting,psychology,electrochemistry,magnetic,anthropology,design,country
3,heart,court,audio,shorter,car,music,light,inventor,chapter,female
4,goal,cream,persuasive,proportion,pro,kinetic,wave,vex,instructor,wages
5,psych,federalism,anna,platform,politics,carboxylic,electricity,combinational,laboratory,reproductive
6,skeletal,supreme,waymaker,justification,beaker,reaction,diffraction,calibration,lecture,decrease
7,speech,ice,spoken,consecutive,ing,resonance,speed,wearable,student,sector
8,summary,bureaucracy,speaking,color,comparative,equilibria,criminal,primate,project,stabilization
9,studio,mike,architectural,calculated,subset,ionic,refraction,resume,policy,inflation


In [47]:
get_lda_topics(lda, 20)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,Topic # 11,Topic # 12,Topic # 13,Topic # 14,Topic # 15,Topic # 16,Topic # 17,Topic # 18,Topic # 19,Topic # 20
0,lab,class,language,calculus,anthropology,art,course,course,government,student,course,due,lab,design,class,course,space,lesson,lesson,analysis
1,course,course,course,cancer,class,works,class,exam,political,class,student,quiz,lecture,lesson,course,academic,aeronautics,body,days,course
2,laboratory,student,culture,smith,human,work,student,college,politics,course,class,complete,course,engineering,economic,student,living,human,activity,exam
3,may,work,communication,use,evolution,content,exam,history,us,instructor,college,course,exam,project,exam,learning,flight,medical,project,composition
4,must,week,exam,treatment,article,ce,may,historical,chapter,policy,instructor,chapter,class,unit,textbook,disability,moon,investigate,data,environmental
5,lecture,please,audio,mike,primate,making,work,score,system,attendance,learning,student,student,course,student,instructor,travel,explore,create,circuit
6,instructor,assignment,interpersonal,physical,physical,artistic,instructor,question,one,may,exam,psychology,may,process,policy,math,history,disease,computer,music
7,chapter,must,presentational,part,course,chemical,use,physics,read,participation,may,discussion,cell,way,history,test,science,health,problem,college
8,exam,grade,college,genetic,skeleton,global,learning,advanced,comparative,academic,use,psych,biology,page,supply,work,traveling,heart,way,science
9,academic,day,study,used,biological,architecture,days,placement,exam,office,information,week,grade,lead,read,policy,test,way,course,writing


In [48]:
get_lda_topics(lda_tfidf, 10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,lesson,constitution,presentational,hybrid,asp,duiker,electric,python,class,terminology
1,architecture,politics,interpretive,mail,marketing,nuclear,magnetism,game,lab,male
2,circuit,cancer,german,painting,psychology,electrochemistry,magnetic,anthropology,design,country
3,heart,court,audio,shorter,car,music,light,inventor,chapter,female
4,goal,cream,persuasive,proportion,pro,kinetic,wave,vex,instructor,wages
5,psych,federalism,anna,platform,politics,carboxylic,electricity,combinational,laboratory,reproductive
6,skeletal,supreme,waymaker,justification,beaker,reaction,diffraction,calibration,lecture,decrease
7,speech,ice,spoken,consecutive,ing,resonance,speed,wearable,student,sector
8,summary,bureaucracy,speaking,color,comparative,equilibria,criminal,primate,project,stabilization
9,studio,mike,architectural,calculated,subset,ionic,refraction,resume,policy,inflation


**HDP**

In [49]:
from gensim.models import HdpModel

In [50]:
def get_hdp_topics(model, num_topics):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = 20);
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict);

In [51]:
hdp = gensim.models.hdpmodel.HdpModel(bow_corpus, id2word=id2word)

In [52]:
hdp_tfidf = gensim.models.hdpmodel.HdpModel(corpus_tfidf, id2word=id2word)

In [53]:
get_hdp_topics(hdp, 10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,class,course,lesson,student,student,course,class,class,lab,course
1,course,student,course,course,course,health,course,course,body,student
2,student,class,project,instructor,class,lesson,student,college,lesson,class
3,exam,may,activity,class,lab,class,exam,student,lecture,communication
4,work,instructor,student,art,instructor,design,may,chapter,exam,may
5,instructor,lab,days,policy,may,student,week,use,explore,work
6,week,academic,data,may,work,material,instructor,computer,human,exam
7,may,exam,create,college,policy,unit,college,may,course,learning
8,grade,policy,problem,academic,time,energy,assignment,exam,structure,instructor
9,history,work,learning,information,academic,power,reading,learning,investigate,college


In [54]:
get_hdp_topics(hdp, 100)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 90,Topic # 91,Topic # 92,Topic # 93,Topic # 94,Topic # 95,Topic # 96,Topic # 97,Topic # 98,Topic # 99
0,class,course,lesson,student,student,course,class,class,lab,course,...,graphical,modernity,formal,embryonic,design,intext,core,anthropology,later,support
1,course,student,course,course,course,health,course,course,body,student,...,twenty,persuasive,disassemble,flowering,delivery,speed,warning,duiker,mitochondria,sacrifice
2,student,class,project,instructor,class,lesson,student,college,lesson,class,...,boat,tab,rock,angle,architectural,laboratory,furniture,resilient,usefulness,governmental
3,exam,may,activity,class,lab,class,exam,student,lecture,communication,...,correlation,creative,presentational,juxtaposition,architecture,observer,otherwise,seeking,di,satisfactory
4,work,instructor,student,art,instructor,design,may,chapter,exam,may,...,transmission,criminal,word,sketching,lesson,analytics,oracle,exacting,fix,drive
5,instructor,lab,days,policy,may,student,week,use,explore,work,...,sky,phase,seventh,footprint,poem,virologist,interval,reasonableness,borer,enlightenment
6,week,academic,data,may,work,material,instructor,computer,human,exam,...,resubmission,allergic,inconsistent,supposed,integer,sport,synchronizer,solvent,distributed,decode
7,may,exam,create,college,policy,unit,college,may,course,learning,...,dynamics,afford,aggregate,doodle,progressively,polymorphism,impacted,vernacular,starting,attentive
8,grade,policy,problem,academic,time,energy,assignment,exam,structure,instructor,...,handicap,learned,beautiful,enable,remarkable,great,overdue,designing,wright,done
9,history,work,learning,information,academic,power,reading,learning,investigate,college,...,definition,scientifically,dissolution,grammatically,high,testimony,gate,prediction,rocket,resultant


In [55]:
get_hdp_topics(hdp_tfig, 10)

NameError: name 'hdp_tfig' is not defined

In [ ]:
get_hdp_topics(hdp_tfig, 100)

Hierarchical Dirichlet Process doesn't seem to work very well 

**New LDA Library**

In [12]:
import lda
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
cvectorizer = CountVectorizer(min_df=5, stop_words='english')
cvz = cvectorizer.fit_transform(cleaned_corpus_3)

In [14]:
n_topics = 20
n_iter = 20



lda_m = lda.LDA(n_topics=n_topics, n_iter=n_iter)
X_topics = lda_m.fit_transform(cvz)

INFO:lda:n_documents: 156
INFO:lda:vocab_size: 2832
INFO:lda:n_words: 125458
INFO:lda:n_topics: 20
INFO:lda:n_iter: 20
C:\Users\Public\Anaconda3\lib\site-packages\lda\utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):
INFO:lda:<0> log likelihood: -1399328
INFO:lda:<10> log likelihood: -969903
INFO:lda:<19> log likelihood: -932084


In [15]:
lda_m.topic_word_

array([[2.09103531e-06, 1.33847170e-02, 2.51133341e-03, ...,
        2.09103531e-06, 2.09103531e-06, 2.09103531e-06],
       [1.73390760e-06, 1.73390760e-06, 2.42920455e-03, ...,
        1.73390760e-06, 1.73390760e-06, 1.73390760e-06],
       [1.91632556e-06, 1.91632556e-06, 1.91632556e-06, ...,
        1.91632556e-06, 1.91632556e-06, 1.91632556e-06],
       ...,
       [1.29595455e-06, 1.29595455e-06, 1.29595455e-06, ...,
        1.29595455e-06, 1.29595455e-06, 1.29595455e-06],
       [1.89669823e-06, 1.89669823e-06, 1.89669823e-06, ...,
        1.89669823e-06, 1.89669823e-06, 1.89669823e-06],
       [1.64871763e-06, 1.64871763e-06, 1.64871763e-06, ...,
        1.64871763e-06, 1.65036635e-03, 3.13421221e-03]])

In [17]:
vocab

In [16]:
 # model.components_ also works
vocab = gensim.corpora.Dictionary(cleaned_corpus_4)    
    
topic_word = lda_m.topic_word_
n_top_words = 8
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(10958 unique tokens: ['academic', 'adaptively', 'addressed', 'advice', 'allowed']...) from 156 documents (total 153738 corpus positions)


TypeError: sequence item 0: expected str instance, numpy.int32 found